In [1]:
# Import Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sc
from bokeh import __version__ as bk_version

In [4]:
# Import the dataset

df = pd.read_csv("Dataset/freMTPL2freq.csv")

df["freq"] = df["ClaimNb"] / df["Exposure"]
df["LogDensity"] = np.log(df["Density"])

print(df.describe)

<bound method NDFrame.describe of             IDpol  ClaimNb  Exposure Area  VehPower  VehAge  DrivAge  \
0             1.0        1   0.10000    D         5       0       55   
1             3.0        1   0.77000    D         5       0       55   
2             5.0        1   0.75000    B         6       2       52   
3            10.0        1   0.09000    B         7       0       46   
4            11.0        1   0.84000    B         7       0       46   
...           ...      ...       ...  ...       ...     ...      ...   
678008  6114326.0        0   0.00274    E         4       0       54   
678009  6114327.0        0   0.00274    E         4       0       41   
678010  6114328.0        0   0.00274    D         6       2       45   
678011  6114329.0        0   0.00274    B         4       0       60   
678012  6114330.0        0   0.00274    B         7       6       29   

        BonusMalus VehBrand   VehGas  Density Region       freq  LogDensity  
0               50     